In [1]:
# https://huggingface.co/docs/peft/index

import os 

### Utils
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def split_strings(strings):
    first_parts = []
    second_parts = []
    for string in strings:
        split_index = len(string) // 2
        while split_index < len(string) and not string[split_index].isspace():
            split_index += 1
        first_part = string[:split_index].rstrip()
        second_part = string[split_index:].lstrip()
        first_parts.append(first_part)
        second_parts.append(second_part)
    return first_parts, second_parts
    
### 

math_example_path = os.getcwd() + '/data/What_Is_Mathematics_An_Elementary_Approach_to_Ideas_and_Methods.txt'

with open(math_example_path, "r",  encoding="utf8") as f:
     math_example_text = f.read()

math_example_text = math_example_text.replace("\n", "")
math_sentences = math_example_text.split(".")

C:\Users\tempdelta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(len(math_sentences))
math_sentences = [item for item in math_sentences if 80 < len(item) < 100]
X, Y = split_strings(math_sentences)
print(math_sentences[1])

9421
 But it is not a concession to the dangerous tendency toward dodging all exertion


In [3]:
tst_ind = 30
print(math_sentences[tst_ind], "\n X: ", X[tst_ind], "\n Y: ", Y[tst_ind])

 For a « 20, we need twenty words for the digits, plus words for 20 and 400, making a total of 22 
 X:   For a « 20, we need twenty words for the digits, 
 Y:  plus words for 20 and 400, making a total of 22


In [34]:
import torch 
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name = "gpt2" #"./finetuned_llm_all" #
tokenizer = GPT2Tokenizer.from_pretrained(model_name, padding_side='left', add_eos_token=True) #gpt2
llm_model = GPT2LMHeadModel.from_pretrained(model_name) # gpt2
device = torch.device('cuda')
llm_model.to(device)

tokenizer.pad_token = tokenizer.eos_token

# Configure the model
llm_model.config.pad_token_id = tokenizer.eos_token_id
llm_model.config.eos_token_id = tokenizer.eos_token_id
llm_model.config.vocab_size = llm_model.config.vocab_size + len(tokenizer.get_added_vocab())
llm_model.resize_token_embeddings(len(tokenizer))
llm_model.config.pad_token_id = tokenizer.pad_token_id

print(tokenizer.eos_token_id)

In [49]:
import numpy as np
#np.random.seed(309)

# Set padding to the left side as the model is a decoder-only architecture
tokenizer.padding_side = "left"

encoded_X = tokenizer.batch_encode_plus(X, add_special_tokens=False, return_tensors='pt', padding=True)
encoded_Y = tokenizer.batch_encode_plus(Y, add_special_tokens=False, return_tensors='pt', padding=True)

BATH_SIZE = 10

x_batches = list(chunks(encoded_X["input_ids"], BATH_SIZE))
y_batches = list(chunks(encoded_Y["input_ids"], BATH_SIZE))

In [50]:
# generate test data
test_ids = np.random.choice(range(len(X)), 2)
test_X = [X[i] for i in test_ids]
encoded_test_X = tokenizer.batch_encode_plus(test_X, 
                                     add_special_tokens=False, 
                                     return_tensors='pt', 
                                     padding=True) # X is encoded Y isnt !


test_Y = [Y[i] for i in test_ids]
###

In [56]:
test_index = 0
test_item = encoded_test_X["input_ids"][test_index]
print(test_item)
print(tokenizer.decode(test_item))

tensor([ 3914,   514,  2074,   262, 20128,   286,   257,  6614])
 Let us consider the projection of a plane


In [57]:
output = llm_model.generate(encoded_test_X["input_ids"].to(device),  max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)

In [58]:
print(output[test_index], "\n ")
print("output : \n", tokenizer.decode(output[test_index]))

tensor([ 3914,   514,  2074,   262, 20128,   286,   257,  6614,   284,   262,
        17810,    13,   198,   198,   464, 20128,   286,   257,  6614,   284,
          262, 17810,   318,   262, 20128,   286,   257,  6614,   284,   262,
        17810,    13,   383, 20128,   286,   257,  6614,   284,   262, 17810,
          318,   262, 20128,   286,   257,  6614,   284,   262, 17810,    13,
          383, 20128,   286,   257,  6614,   284,   262, 17810,   318,   262,
        20128,   286,   257,  6614,   284,   262, 17810,    13,   383, 20128,
          286,   257,  6614,   284,   262, 17810,   318,   262, 20128,   286,
          257,  6614,   284,   262, 17810,    13,   383, 20128,   286,   257,
         6614,   284,   262, 17810,   318,   262, 20128,   286,   257,  6614,
          284,   262, 17810,    13,   383, 20128,   286,   257,  6614,   284,
          262, 17810,   318,   262, 20128,   286,   257,  6614,   284,   262,
        17810,    13,   383, 20128,   286,   257,  6614,   284, 

In [ ]:
# Prepare input data

tst_i = 3
test_str = test_X[tst_i]
input_ids = tokenizer.encode(test_str,
                             add_special_tokens=False, 
                             return_tensors='pt', 
                             padding=True).to(device)

# Generate text using generation parameters
output = llm_model.generate(input_ids , 
                            max_new_tokens=100, 
                            pad_token_id=tokenizer.eos_token_id)

# Decode and print the output
decoded_output = tokenizer.decode(output[0])
print(f"input: {test_str} \n")
print(f"input_ids: {input_ids} \n")
print(f"output: {output} \n")

print(f"decoded_output:{decoded_output}\n", )
print(f"expected output:{test_Y[tst_i]} ")

In [ ]:
import torch

optimizer = torch.optim.AdamW(llm_model.parameters(), lr=1e-5)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
import mlflow


EPOCHS=10

mlflow.end_run()
gradient_accumulation_steps = 10
grad_counter = 0

with mlflow.start_run(run_name='final') as run:

    for epoch in range(EPOCHS):
        for index, batch in enumerate(x_batches[0:5]): 
            outputs = llm_model(batch, labels=y_batches[index])
            loss = outputs.loss
            loss.backward()

            if grad_counter % gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

                print("batch loss ", loss.item())

                
            grad_counter += 1
                            

        # log metric every epoch
        mlflow.log_metric('loss', loss.item())
        mlflow.log_metric('lr', get_lr(optimizer))

        
        output = llm_model.generate(tst_data["input_ids"].to(device))
        test_output = ""
        for i in range(output.shape[0]):
            tmp_str = tokenizer.decode(output[i])
            test_output += "\n" + tmp_str
        mlflow.log_text(test_output, f"test_{epoch}.txt")
        #### 

        
        # Save the fine-tuned model every epoch 
        print("MODEL saved loss ", loss)
        llm_model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

In [ ]:
import numpy as np
tst_text = np.random.choice(math_sentences, 30)
tst_data = tokenizer.batch_encode_plus(tst_text, add_special_tokens=True, return_tensors='pt', padding=True).to(device)
output = llm_model.generate(tst_data["input_ids"])

for i, text in enumerate(tst_text):
    print("input:", text)
    print("output", tokenizer.decode(output[i]))

In [ ]:
from transformers import TextDataset

train_dataset = TextDataset(
      tokenizer=tokenizer,
      file_path=math_example_path,
      block_size=128)

In [ ]:
from transformers import (
    TrainingArguments, 
    Trainer, 
    default_data_collator, 
)
import mlflow 
mlflow.end_run()

training_args = TrainingArguments(output_dir='test_trainer', 
                                  #evaluation_strategy='epoch',
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=1,
                                  gradient_accumulation_steps=20, #
                                  num_train_epochs = 2,
                                  fp16=False,)

trainer = Trainer(
    model=llm_model,
    args=training_args,
    train_dataset=dataset,
    #eval_dataset=dataset,
    tokenizer=tokenizer,
    # Data collator will default to DataCollatorWithPadding, so we change it.
    data_collator=default_data_collator,
    compute_metrics=None,
    preprocess_logits_for_metrics=None,
)

trainer.train()

In [ ]:
dataset